In [17]:
import pandas as pd

In [18]:
# url = "https://econdata.s3-us-west-2.amazonaws.com/Reports/Core/RDC_Inventory_Core_Metrics_State.csv"
# df = pd.read_csv(url)
# df.to_csv("data1.csv" , index=False)

In [19]:
from supabase import create_client
import os

# Load environment variables
from dotenv import load_dotenv
load_dotenv()

SUPABASE_URL = os.getenv("SUPABASE_URL")
SUPABASE_KEY = os.getenv("SUPABASE_SERVICE_ROLE_KEY")

# Push preprocessed data to Supabase (state_market table)
supabase = create_client(SUPABASE_URL, SUPABASE_KEY)
from get_supabase_data import get_supabase_data
existing_data = get_supabase_data("state_market") 

In [20]:
from preprocessing_2 import preprocess_data_2
from model_trainer import get_predictions

target_df = preprocess_data_2(existing_data.copy())
prediction_df = target_df.copy()
prediction_df

,year,month,state_num
0,2025,10,1
1,2025,10,2
2,2025,10,3
3,2025,10,4
4,2025,10,5
...,...,...,...
148,2025,12,47
149,2025,12,48
150,2025,12,49
151,2025,12,50


In [21]:
features = [
    "median_listing_price",
    "average_listing_price",
    "median_listing_price_per_square_foot",
    "total_listing_count",
    "median_days_on_market"
]

for feature in features:
    predictions = get_predictions(existing_data, feature)
    prediction_df[feature] = predictions


Validation RMSE: 5043.8624, RMSLE: 0.0101, MAE: 3942.8045, MAPE: 0.85%, R²: 0.9987
Validation RMSE: 10103.4091, RMSLE: 0.0212, MAE: 7578.1832, MAPE: 1.66%, R²: 0.9944
Validation RMSE: 9009.4267, RMSLE: 0.0182, MAE: 6593.4409, MAPE: 1.46%, R²: 0.9956
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.024297 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 250260
[LightGBM] [Info] Number of data points in the train set: 3570, number of used features: 998
[LightGBM] [Info] Start training from score 0.638591
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.035134 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 250260
[LightGBM] [Info] Number of data points in the train set: 3570, number of used features: 998
[LightGBM] [Info] Start training from score 0.647893
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the 

In [22]:
for col in features:
    if col in prediction_df.columns:
        prediction_df[col] = prediction_df[col].astype(int)

In [23]:
prediction_df.isnull().sum()

year                                    0
month                                   0
state_num                               0
median_listing_price                    0
average_listing_price                   0
median_listing_price_per_square_foot    0
total_listing_count                     0
median_days_on_market                   0
dtype: int64

In [28]:
data_to_insert = prediction_df.to_dict('records')

In [39]:
supabase.table("predictions").delete().gt("year", 0).execute()

APIResponse[~_ReturnT](data=[{'year': 2025, 'month': 10, 'state_num': 1, 'median_listing_price': 329239, 'average_listing_price': 438233, 'median_listing_price_per_square_foot': 165, 'total_listing_count': 27945, 'median_days_on_market': 71, 'market_trend': None}, {'year': 2025, 'month': 10, 'state_num': 2, 'median_listing_price': 439403, 'average_listing_price': 530773, 'median_listing_price_per_square_foot': 253, 'total_listing_count': 2892, 'median_days_on_market': 67, 'market_trend': None}, {'year': 2025, 'month': 10, 'state_num': 3, 'median_listing_price': 481124, 'average_listing_price': 728124, 'median_listing_price_per_square_foot': 265, 'total_listing_count': 36854, 'median_days_on_market': 61, 'market_trend': None}, {'year': 2025, 'month': 10, 'state_num': 4, 'median_listing_price': 299015, 'average_listing_price': 398900, 'median_listing_price_per_square_foot': 160, 'total_listing_count': 17403, 'median_days_on_market': 70, 'market_trend': None}, {'year': 2025, 'month': 10, 

In [38]:
supabase.table("predictions").insert(data_to_insert).execute()

APIResponse[~_ReturnT](data=[{'year': 2025, 'month': 10, 'state_num': 1, 'median_listing_price': 329239, 'average_listing_price': 438233, 'median_listing_price_per_square_foot': 165, 'total_listing_count': 27945, 'median_days_on_market': 71, 'market_trend': None}, {'year': 2025, 'month': 10, 'state_num': 2, 'median_listing_price': 439403, 'average_listing_price': 530773, 'median_listing_price_per_square_foot': 253, 'total_listing_count': 2892, 'median_days_on_market': 67, 'market_trend': None}, {'year': 2025, 'month': 10, 'state_num': 3, 'median_listing_price': 481124, 'average_listing_price': 728124, 'median_listing_price_per_square_foot': 265, 'total_listing_count': 36854, 'median_days_on_market': 61, 'market_trend': None}, {'year': 2025, 'month': 10, 'state_num': 4, 'median_listing_price': 299015, 'average_listing_price': 398900, 'median_listing_price_per_square_foot': 160, 'total_listing_count': 17403, 'median_days_on_market': 70, 'market_trend': None}, {'year': 2025, 'month': 10, 